In [0]:
#Tratamiento de datos
import pandas as pd
import numpy as np

#Graficos
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
import plotly.express as px
plt.style.use('fivethirtyeight')
plt.rcParams['lines.linewidth'] = 1.5
%matplotlib inline

# Modelado y Forescasting
#from lightgbm import LGBMRegressor    #Regressor mas potente de gradient bossting
# from skforecast.ForecasterAutoreg import ForecasterAutoreg  # Modelo autoregressivo
# from skforecast.model_selection import grid_search_forecaster # Encontrar los mejores hiperparametros(optimizar el modelo)
# from skforecast.model_selection import backtesting_forecaster  # Evaluar el modelo si hubiese estado en produccion
from pyspark.sql.functions import col

#Configuration warning
import warnings
warnings.filterwarnings('ignore')

In [0]:
#Librerias que se usaran
import pandas as pd  # Libreria para administrar tablas, y realizar trabajos con distintas formas de tablas o dataframes
import numpy as np   # Libreria para poder hacer operaciones matematicas y matriciales 
import pandas as pd
from pyspark.sql.functions import col

#1. Leemos los datos de TABLA CARACTERISTICAS la tabla Delta de storage Presentation 
#df_delta = spark.read.format("delta").load("/mnt/datalakemlopsd4m/presentation/proyectocongestion_presentation/#tablacaracteristicas_congestion_tabladelta")
df_delta = spark.read.format("delta").load("/mnt/datalakemlopsd4m/processed/proyectocongestion_processed/datapreprocessed_congestion_tabladelta")

datos = df_delta.toPandas()

datos.head()

In [0]:
#Configuramos pandas para que podamos vizualizar todas las columnas y filas la estadistica descriptiva de todas las variables
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
#Configuramos pandas para que lanze valores con una precision de hasta 6 decimales
pd.set_option('display.float_format', '{:.6f}'.format)

In [0]:
datos.dtypes

#### 1. Ajuste de los datos a Series Temporales

In [0]:
#2. Ajuste de los datos a Series Temporales
#2.1 Se establece la columna 'Time' como el índice del DataFrame "datos"
datos = datos.set_index('instant_date_t')

#2.3 Ordenamos el dataset de forma ascendente segun el datetime
datos.sort_index(inplace=True)

#2.4 Identificamos la periocidad de la serie temporal
df_time_diffs = datos.index.to_series().diff().dt.total_seconds()

#3. Eliminamos o Filtramos las filas donde la diferencia es distinta de cero (Con ello eliminamos las filas o registros de fechas duplicadas)
datos = datos[df_time_diffs != 0]

#4. # Reinterpolar el dataset con una periosidad en especifico
# 'S' o 'seg' : Segundo, 'T' o 'min': Minuto,'H' o 'Hr': Hora,'D': Día,'W': Semana,'M': Mes,'Q': Trimestre,'Y': Año
datos = datos.asfreq(freq='T', method='bfill')

In [0]:
datos.head()

#### 2. Separacion de los datos en Train-Validation-Test

In [0]:
# Obtener la fecha mínima y máxima del índice
fecha_minima = datos.index.min()
fecha_maxima = datos.index.max()

# Imprimir los resultados
print("Fecha mínima:", fecha_minima)
print("Fecha máxima:", fecha_maxima)

In [0]:
#Separacion de los datos en Train-Validation-Test
datos = datos.loc[fecha_minima: fecha_maxima]
fin_train = '2024-03-01 18:47:18'
fin_validacion = '2024-03-01 21:47:18'
datos_train = datos.loc[:fin_train, :] #Un año
datos_val = datos.loc[fin_train:fin_validacion, :] #2 meses
datos_test = datos.loc[fin_validacion: , :] # 3 meses

print(f"Fechas train: {datos_train.index.min()} ... {datos_train.index.max()} (n={len(datos_train)})")
print(f"Fechas validacion: {datos_val.index.min()} ... {datos_val.index.max()} (n={len(datos_val)})")
print(f"Fechas test: {datos_test.index.min()} ... {datos_test.index.max()} (n={len(datos_test)})")

In [0]:
print(datos_train.shape,datos_val.shape, datos_test.shape)

#### 3.Grafico de la serie temporal (train,validation, test)

In [0]:
#Grafico de la serie temporal

#crea una nueva figura y un par de ejes (axes) dentro de la figura
fig, ax = plt.subplots(figsize=(15,4)) # figsize=(15,4) establece el tamaño de la figura en 15 unidades de ancho y 4 unidades de alto.

#Las series se colorean automáticamente según una paleta de colores predefinida por matplotlib
datos_train.speed_t.plot(ax=ax, label="entrenamiento", linewidth=1)
datos_val.speed_t.plot(ax=ax, label="validacion", linewidth=1)
datos_test.speed_t.plot(ax=ax, label="test", linewidth=1)
ax.set_title('speed_t')
ax.legend();